# Notebook to download Sentinel-2 Satellite GeoTIFF images of forested areas in California after documented wildfires using Google Earth Engine (GEE)
## The data points and polygons were gathered from the folowing [Esri Living Atlas Layer ](https://pns.maps.arcgis.com/home/item.html?id=6fd0d8d6f47d414da7bcb1dcd0539999)
- The layer was preprocessed in ArcGIS Pro:
    - The layer was filtered to get dates from 2016 because we plan on using sentinel.
    - The layer was clipped using the Forest Service of the Original Proclaimed National Forest Land layer.
    - The Feature to point tool was used.
    - The Get XY point tool was applied.
    - The final layer attribute table was saved as an excel file "fireForest.xls"

#### Import packages and libraries

In [1]:
#Import the necessary libraries and packages
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pyproj
import warnings
warnings.filterwarnings('ignore')
from shapely.geometry import Point, Polygon
from geopy.distance import great_circle
import ee
import geemap
import numpy as np
import geetools
import time

## Define the processed files to be used and the after dates

In [2]:
excel_file_path = 'ExcelFiles/DataFireFinal.xlsx'  
df = pd.read_excel(excel_file_path)
# Drop the specified columns
columns_to_drop = ['15_Days_Before_Incident', '1_Day_Before_Incident', 'Incident_Date', 'Containment_Date']
df = df.drop(columns=columns_to_drop)

# Drop rows where '1_Day_After_Containment' or '15_Days_After_Containment' is empty

df = df.dropna(subset=['1_Day_After_Containment', '15_Days_After_Containment'])

# Convert the date columns to datetime if they are not already in datetime format
df['1_Day_After_Containment'] = pd.to_datetime(df['1_Day_After_Containment'])
df['15_Days_After_Containment'] = pd.to_datetime(df['15_Days_After_Containment'])

# Drop rows where either '1_Day_After_Containment' or '15_Days_After_Containment' is '2020-01-01'
df = df.loc[~((df['1_Day_After_Containment'] == '2020-01-01') | (df['15_Days_After_Containment'] == '2020-01-01'))]

# Revert the datetime columns back to their original format (as strings)
df['1_Day_After_Containment'] = df['1_Day_After_Containment'].dt.strftime('%Y-%m-%d')
df['15_Days_After_Containment'] = df['15_Days_After_Containment'].dt.strftime('%Y-%m-%d')

dataAfter = df.values


## Download the After Wildfire RGB and NDVI images using GEE

### Install and authenticate GEE

In [3]:

#Authenticate 
ee.Authenticate()

#initialize ee
ee.Initialize()


Enter verification code:  4/1AfJohXkwhH13QN6J3ZObpOLliw59cHD6jwBlgwtP9qgB524rAeLsJTIY19I



Successfully saved authorization token.


### Export the RGB and NDVI GeoTIFFs (B2, B3, B4, and B8) Images for After the fire event using GEE.

In [20]:
num = []
for i in range(331, 1000):
    num.append(i)

# Define the bands you want to download
bands_rgb = ['B4', 'B3', 'B2']  # Red, Green, Blue
bands_ndvi = ['B8', 'B4']  # NIR, Red

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

for i in num:
    # Set the center point
    point = ee.Geometry.Point(round(dataAfter[i][1], 2), round(dataAfter[i][2], 2))

    # Call an image for the selected point
    tile = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(point) \
        .filterDate(dataAfter[i][3], dataAfter[i][4]) \
        .sort('CLOUDY_PIXEL_PERCENTAGE') \
        .first()
    # Check the properties of the image
    image_properties = tile.getInfo()
    cloudy_percentage = image_properties.get('properties', {}).get('CLOUDY_PIXEL_PERCENTAGE', 0)
    if cloudy_percentage <= 10:
        # Define the ROI
        roi = ee.Geometry.Polygon([[
            [dataAfter[i][5], dataAfter[i][6]],
            [dataAfter[i][7], dataAfter[i][8]],
            [dataAfter[i][9], dataAfter[i][10]],
            [dataAfter[i][11], dataAfter[i][12]],
            [dataAfter[i][13], dataAfter[i][14]]]])

        # Add NDVI to the selected bands
        tile_with_ndvi = calculate_ndvi(tile)

        # Select RGB and NDVI bands
        rgb_tile = tile.select(bands_rgb)
        ndvi_tile = tile_with_ndvi.select(['NDVI'])

        # Export RGB image to Google Drive
        rgb_task = ee.batch.Export.image.toDrive(**{
            'image': rgb_tile,
            'description': 'RGB_AfterFire' + str(dataAfter[i][0]),
            'folder': 'GEE_FireImagesRGB_after',
            'scale': 10,  # Adjust the scale as needed
            'region': roi.getInfo()['coordinates'],
            'crs': 'EPSG:4326',
            'fileFormat': 'GeoTIFF',
        })
        rgb_task.start()

        # Export NDVI image to Google Drive
        ndvi_task = ee.batch.Export.image.toDrive(**{
            'image': ndvi_tile,
            'description': 'NDVI_AfterFire' + str(dataAfter[i][0]),
            'folder': 'GEE_FireImagesNDVI_after',
            'scale': 10,  # Adjust the scale as needed
            'region': roi.getInfo()['coordinates'],
            'crs': 'EPSG:4326',
            'fileFormat': 'GeoTIFF',
        })
        ndvi_task.start()

        # Wait for the tasks to complete
        while not (rgb_task.status()['state'] in ['COMPLETED', 'FAILED', 'CANCELLED'] and
                   ndvi_task.status()['state'] in ['COMPLETED', 'FAILED', 'CANCELLED']):
            print('RGB Task is', rgb_task.status()['state'])
            print('NDVI Task is', ndvi_task.status()['state'])
            time.sleep(90)  # Wait for 90 seconds before checking again

        # Check the final task statuses
        rgb_task_status = rgb_task.status()
        ndvi_task_status = ndvi_task.status()
        print("RGB Task Status:", rgb_task_status)
        print("NDVI Task Status:", ndvi_task_status)
        print("RGB Task Error Message:", rgb_task_status.get("error_message", "No error message"))
        print("NDVI Task Error Message:", ndvi_task_status.get("error_message", "No error message"))
    else:
        print(f"Skipping image {i} due to cloudy percentage ({cloudy_percentage}%) > 10%")
    

RGB Task is READY
NDVI Task is READY
RGB Task is READY
NDVI Task is READY
RGB Task is READY
NDVI Task is READY
RGB Task is RUNNING
NDVI Task is RUNNING
RGB Task is RUNNING
NDVI Task is RUNNING
RGB Task is RUNNING
NDVI Task is RUNNING
RGB Task Status: {'state': 'COMPLETED', 'description': 'RGB_AfterFire464', 'creation_timestamp_ms': 1700803626453, 'update_timestamp_ms': 1700804150179, 'start_timestamp_ms': 1700803888768, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1zd9Wx7GXqw-w9F4hDZyViF9Rq04K5veI'], 'attempt': 1, 'batch_eecu_usage_seconds': 21.219398498535156, 'id': 'YXTRRLLKHTLN77OQEEN5WRA4', 'name': 'projects/earthengine-legacy/operations/YXTRRLLKHTLN77OQEEN5WRA4'}
NDVI Task Status: {'state': 'COMPLETED', 'description': 'NDVI_AfterFire464', 'creation_timestamp_ms': 1700803627249, 'update_timestamp_ms': 1700804156279, 'start_timestamp_ms': 1700803896421, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1uXzVE

IndexError: index 572 is out of bounds for axis 0 with size 572